# Evaluate test performance for object detection task

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Get Data

In [0]:
%matplotlib inline
import os
import time

import skimage 
from skimage.transform import resize
from skimage.exposure import rescale_intensity
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import PIL

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset as torchDataset
import torchvision as tv
from torch.autograd import Variable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Rectangle

import shutil

!pip install pydicom --quiet
import pydicom

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.catch_warnings()

gpu_available = True

original_image_shape = 1024

datapath_orig = '/content/3. Object Identification/'
datapath_prep = '/content/gdrive/My Drive/Hackathon/'
datapath_out = '/content/gdrive/My Drive/Hackathon/'

     |████████████████████████████████| 35.3MB 131kB/s 


# Helper Functions

In [0]:
def box_mask(box, shape=1024):
    """
    :param box: [x, y, w, h] box coordinates
    :param shape: shape of the image (default set to maximum possible value, set to smaller to save memory)
    :returns: (np.array of bool) mask as binary 2D array
    """
    x, y, w, h = box
    mask = np.zeros((shape, shape), dtype=bool)
    mask[int(y):int(y+h), int(x):int(x+w)] = True 
    return mask

def IoU(pr, gt):
    """
    :param pr: (numpy_array(bool)) prediction array 
    :param gt: (numpy_array(bool)) ground truth array 
    :returns: IoU (pr, gt) = intersection (pr, gt) / union (pr, gt)
    """
    IoU = (pr & gt).sum() / ((pr | gt).sum() + 1.e-9)
    return IoU

# Hits calculate function

In [0]:
def hits(preds_test, ground_truth, shape=1024):
    correct = 0
    total = len(ground_truth)

    for index, pred in enumerate(preds_test):
      if pred == [] and ground_truth[index] == []:
        correct+=1
      else:
          if len(pred)>0 and ground_truth[index] == []:
              pass
          elif len(ground_truth[index])>0 and pred == []:
              pass
          else:
            found = False
            for box_p in pred: 
                      box_p_msk = box_mask(box_p, shape) 
                      for box_t in ground_truth[index]: 
                          box_t_msk = box_mask(box_t, shape) 
                          iou = IoU(box_p_msk, box_t_msk) 
                          if iou>0.5:
                            found = True
            if found:
              correct +=1
    return correct/total
            

# Calculate IOU hits

In [0]:
def IoU(pr, gt):
    """
    :param pr: (numpy_array(bool)) prediction array 
    :param gt: (numpy_array(bool)) ground truth array 
    :returns: IoU (pr, gt) = intersection (pr, gt) / union (pr, gt)
    """
    IoU = (pr & gt).sum() / ((pr | gt).sum() + 1.e-9)
    return IoU

In [0]:
df_test = pd.read_csv("/content/gdrive/My Drive/Hackathon/test.csv")
df_test.head()

,patientId,x,y,width,height,Target,class,PatientSex,PatientAge,ViewPosition
0,7c2ad76c-88a5-4a25-a0e4-49437ea5f628,NaN,NaN,NaN,NaN,0,Normal,M,73.0,PA
1,503cee4f-1ffe-478e-b3b7-0ced2ae1d5b0,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal,M,60.0,PA
2,8a2ff8f3-6c66-4d64-9fd1-59fc17c2d59e,542.0,87.0,270.0,703.0,1,Lung Opacity,M,32.0,AP
3,73176c86-28cb-425b-9744-27b76446aabf,NaN,NaN,NaN,NaN,0,Normal,M,34.0,PA
4,f75f713d-fbf5-4844-a112-f8d0e6683ff9,NaN,NaN,NaN,NaN,0,Normal,M,59.0,PA


# Check test performance

In [0]:
box_patients = df_test.dropna()
ground_truth = []
preds = []

test_ids = list(df_test['patientId'])

for id in test_ids:
  try:
    info = box_patients.loc[box_patients['patientId'] == id].loc[:, ['x','y','width','height']]
    ground_truth.append(info.values.tolist())
  except KeyError:
    ground_truth.append([])

In [0]:
import pickle
preds_test = pickle.load( open('/content/gdrive/My Drive/Hackathon/preds_boxes.pkl', 'rb'))

In [0]:
print("Hit percent : ", hits(preds_test, ground_truth)*100, "%")

Hit percent :  40.505491597194656 %
